In [1]:
from omegaconf import OmegaConf
import os
import torch

import hydra
from omegaconf import DictConfig, OmegaConf
from models.together_model import ProteinVAELLMmodel
from data import all_atom
# Pytorch lightning imports
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

from data.pdb_dataloader import PdbDataModule
from models.flow_module import FlowModule
from experiments import utils as eu
import openfold.utils.rigid_utils as ru
import torch.nn.functional as F
cfg = OmegaConf.load("configs/base.yaml")
_cfg = cfg
_data_cfg = cfg.data
_exp_cfg = cfg.experiment
_datamodule: LightningDataModule = PdbDataModule(_data_cfg)
_datamodule.setup(stage="fit")
torch.autograd.set_detect_anomaly(True)

/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set stuff up

In [2]:
cfg = OmegaConf.load("configs/base.yaml")
_cfg = cfg
_data_cfg = cfg.data
_exp_cfg = cfg.experiment
training_cfg = _exp_cfg.training
_datamodule: LightningDataModule = PdbDataModule(_data_cfg)
_datamodule.setup(stage="fit")

In [3]:
train_loader = _datamodule.train_dataloader()

/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
from data.my_interpolant import Interpolant 
interpolant = Interpolant(cfg.interpolant)
interpolant.set_device("cuda")

In [5]:
model = ProteinVAELLMmodel(cfg).to("cuda")
model.attach_backward_hooks() # attach backward hooks

In [6]:
optimizer = torch.optim.AdamW(
            params=model.parameters(),
            lr=0.0001)

In [7]:
model

ProteinVAELLMmodel(
  (framediff_model): FlowModel(
    (node_embedder): NodeEmbedder(
      (linear): Linear(in_features=256, out_features=128, bias=True)
    )
    (edge_embedder): EdgeEmbedder(
      (linear_s_p): Linear(in_features=128, out_features=64, bias=True)
      (linear_relpos): Linear(in_features=64, out_features=64, bias=True)
      (edge_embedder): Sequential(
        (0): Linear(in_features=236, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
        (4): Linear(in_features=64, out_features=64, bias=True)
        (5): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (trunk): ModuleDict(
      (ipa_0): InvariantPointAttention(
        (linear_q): Linear(in_features=128, out_features=1024, bias=True)
        (linear_kv): Linear(in_features=128, out_features=2048, bias=True)
        (linear_q_points): Linear(in_features=128, out_features=192, bias=True)
        (linear

# Train stuff

In [8]:
batch = next(iter(train_loader))
print(batch["aatype"])
for key, value in batch.items():
    batch[key] = value.to("cuda")
noisy_batch = interpolant.corrupt_batch(batch)

tensor([[10,  1, 14, 13,  2, 10, 19,  9, 14, 13,  0, 19,  5, 11,  7,  6, 10, 16,
          7,  6, 19,  1, 12, 14, 15,  7, 11, 16,  0,  1, 14,  2,  9, 16,  3,  2,
         11,  3,  7, 16,  9, 16, 19,  1, 18,  0, 14, 16,  6, 11,  7, 10,  8,  5,
         12,  7,  9, 11, 18,  3,  7,  2,  8,  9, 14,  7, 15, 14, 10,  5, 13, 18,
         19,  3,  0,  9,  2, 15,  1,  8],
        [ 6, 14, 19, 15, 11, 10,  1,  9,  1, 16, 14, 15,  7,  6, 13, 10,  6,  1,
          1, 13, 10,  0, 15,  2, 11, 10,  5,  9, 19, 13,  3, 13, 19,  0, 15, 11,
          7, 13, 14, 17,  3,  6, 18, 11, 10, 10, 15, 16, 13, 14,  1,  1,  3, 19,
         16,  5, 10,  3, 14,  2, 11, 15, 10, 10,  6, 19, 11, 10, 13, 14,  5,  6,
         16, 10, 13, 10,  6,  0, 11,  6],
        [16,  6,  6,  5, 11,  1, 12, 10, 10, 10, 14,  0,  6,  0, 19,  2, 19, 15,
          1, 11,  7,  6, 13, 16, 10, 11, 19,  7,  7, 15, 10, 11,  7,  0, 11,  2,
         19, 18, 18,  2, 12,  0, 10, 12,  2,  0,  7, 19, 11, 11, 19, 19, 19,  1,
         13,  3, 14,  5, 

In [9]:
B, l, num_res = noisy_batch['res_mask'].shape
loss_mask = noisy_batch['res_mask'].reshape(B*l, num_res)
        
if training_cfg.min_plddt_mask is not None:
    plddt_mask = noisy_batch['res_plddt'] > training_cfg.min_plddt_mask
    loss_mask *= plddt_mask
        

# Ground truth labels
gt_trans = noisy_batch['trans_t']
gt_rotmats = noisy_batch['rotmats_t'] 

# Model output predictions.

framediff_out = model(noisy_batch)
pred_trans = framediff_out["pred_T"]['pred_trans'].reshape(B, l, 128, 3)
pred_rotmats = framediff_out["pred_T"]['pred_rotmats'].reshape(B, l, 128, 3, 3)
        
# Shift for CausalLM
shifted_pred_trans  = pred_trans[:, :-1, :, :]
shifted_pred_rotmats = pred_rotmats[:, :-1, :, :, :]
shifted_gt_trans = gt_trans[:, 1:, :, :]
shifted_gt_rotmats = gt_rotmats[:, 1:, :, :, :]

# Reshape back to [B*(l-1), 128, *]
flat_shifted_pred_trans = shifted_pred_trans.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3)
flat_shifted_pred_rotmats = shifted_pred_rotmats.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3, 3)

flat_shifted_gt_trans = shifted_gt_trans.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3)
flat_shifted_gt_rotmats = shifted_gt_rotmats.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3, 3)

# Timestep used for normalization.
t = noisy_batch['t'].reshape(B, l, 1)[:, 1:, :].reshape(-1,1) # We throw away the first time points
norm_scale = 1 - torch.min(
    t[..., None], torch.tensor(training_cfg.t_normalize_clip))
        
        

# Backbone atom loss
gt_bb_atoms = all_atom.to_atom37(flat_shifted_gt_trans, flat_shifted_gt_rotmats)[:, :, :3] 
pred_bb_atoms = all_atom.to_atom37(flat_shifted_pred_trans, flat_shifted_pred_rotmats)[:, :, :3]

gt_bb_atoms *= training_cfg.bb_atom_scale / norm_scale[..., None]
pred_bb_atoms *= training_cfg.bb_atom_scale / norm_scale[..., None]
                
        
loss_denom = torch.sum(loss_mask, dim=-1, dtype=torch.float).mean() * 3 # Added a mean here, this doesn'y matter since our mask is all 1's
bb_atom_loss = torch.sum(
    (gt_bb_atoms - pred_bb_atoms) ** 2,
    dim=(-1, -2, -3)
) / loss_denom

# Pairwise distance loss
num_batch = gt_bb_atoms.shape[0]
gt_flat_atoms = gt_bb_atoms.reshape([num_batch, num_res*3, 3])
gt_pair_dists = torch.linalg.norm(
    gt_flat_atoms[:, :, None, :] - gt_flat_atoms[:, None, :, :], dim=-1)
pred_flat_atoms = pred_bb_atoms.reshape([num_batch, num_res*3, 3])
pred_pair_dists = torch.linalg.norm(
    pred_flat_atoms[:, :, None, :] - pred_flat_atoms[:, None, :, :], dim=-1)

flat_loss_mask = torch.tile(loss_mask[:, :, None], (1, 1, 3))[B:,:,:] # change the shape because we shifted tokens, all entries of loss masks are 1 so don't matter, we throw away B tokens
flat_loss_mask = flat_loss_mask.reshape([num_batch, num_res*3])
flat_res_mask = torch.tile(loss_mask[:, :, None], (1, 1, 3))[B:,:,:] # change the shape because we shifted tokens, all entries of loss masks are 1 so don't matter
flat_res_mask = flat_res_mask.reshape([num_batch, num_res*3])

gt_pair_dists = gt_pair_dists * flat_loss_mask[..., None]
pred_pair_dists = pred_pair_dists * flat_loss_mask[..., None]
pair_dist_mask = flat_loss_mask[..., None] * flat_res_mask[:, None, :]

dist_mat_loss = torch.sum(
    (gt_pair_dists - pred_pair_dists)**2 * pair_dist_mask,
    dim=(1, 2))
dist_mat_loss /= (torch.sum(pair_dist_mask, dim=(1, 2)) - num_res)

auxiliary_loss = (bb_atom_loss + dist_mat_loss) * (
    t[:, 0]> training_cfg.aux_loss_t_pass
)
auxiliary_loss *= _exp_cfg.training.aux_loss_weight
        
auxiliary_loss = auxiliary_loss.mean()
kl_div = (1 + 2 * framediff_out["vae_log_sigma"] - framediff_out["vae_mu"].pow(2) - framediff_out["vae_log_sigma"].exp().pow(2))[:, :-1, :] # Throw away the last mu and sigma because we're not using it to predict
kl_div = - 0.5 * kl_div.sum(dim=-1).mean()
mse_loss = F.mse_loss(flat_shifted_pred_trans, flat_shifted_gt_trans) + F.mse_loss(flat_shifted_pred_rotmats, flat_shifted_gt_rotmats)

Parameters with NaNs: []
Parameters without NaNs: ['framediff_model.node_embedder.linear.weight', 'framediff_model.node_embedder.linear.bias', 'framediff_model.edge_embedder.linear_s_p.weight', 'framediff_model.edge_embedder.linear_s_p.bias', 'framediff_model.edge_embedder.linear_relpos.weight', 'framediff_model.edge_embedder.linear_relpos.bias', 'framediff_model.edge_embedder.edge_embedder.0.weight', 'framediff_model.edge_embedder.edge_embedder.0.bias', 'framediff_model.edge_embedder.edge_embedder.2.weight', 'framediff_model.edge_embedder.edge_embedder.2.bias', 'framediff_model.edge_embedder.edge_embedder.4.weight', 'framediff_model.edge_embedder.edge_embedder.4.bias', 'framediff_model.edge_embedder.edge_embedder.5.weight', 'framediff_model.edge_embedder.edge_embedder.5.bias', 'framediff_model.trunk.ipa_0.head_weights', 'framediff_model.trunk.ipa_0.linear_q.weight', 'framediff_model.trunk.ipa_0.linear_q.bias', 'framediff_model.trunk.ipa_0.linear_kv.weight', 'framediff_model.trunk.ipa_

/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/torch/nn/modules/module.py:1554: UserWarning: For backward hooks to be called, module output should be a Tensor or a tuple of Tensors but received <class 'transformers.modeling_outputs.BaseModelOutputWithPastAndCrossAttentions'>
  warnings.warn("For backward hooks to be called,"
/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/torch/nn/modules/module.py:1554: UserWarning: For backward hooks to be called, module output should be a Tensor or a tuple of Tensors but received <class 'dict'>
  warnings.warn("For backward hooks to be called,"


In [10]:
loss = mse_loss + auxiliary_loss + kl_div
print(loss)
optimizer.zero_grad()
loss.backward()
optimizer.step()

tensor(48640104., device='cuda:0', grad_fn=<AddBackward0>)


/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: Error detected in LinalgEighBackward0. Traceback of forward call that caused the error:
  File "/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start
    self.io_loop.start()
  File "/gpfs/gibbs/project/d

RuntimeError: Function 'LinalgEighBackward0' returned nan values in its 0th output.